# ML Approach 


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage import feature, color
import os

# Function to extract HOG features from an image
def extract_hog_features(image):
    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)
    # Extract HOG features
    hog_features = feature.hog(gray_image)
    return hog_features

# Function to load and preprocess the dataset
def load_dataset(data_dir):
    # Initialize lists to store features and labels
    features = []
    labels = []

    # Loop through each person in the dataset
    for person_id in range(1, 31):
        # Load genuine signatures
        for i in range(1, 6):
            image_path = os.path.join(data_dir, f'person_{person_id}_genuine_{i}.jpg')
            signature_image = load_and_preprocess_image(image_path)
            hog_features = extract_hog_features(signature_image)
            features.append(hog_features)
            labels.append(0)  # Label 0 for genuine signatures

        # Load forged signatures   #mtdeesh kolo real then kolo forged
        for i in range(1, 6):
            image_path = os.path.join(data_dir, f'person_{person_id}_forged_{i}.jpg')
            signature_image = load_and_preprocess_image(image_path)
            hog_features = extract_hog_features(signature_image)
            features.append(hog_features)
            labels.append(1)  # Label 1 for forged signatures

    return np.array(features), np.array(labels)

# Function to load and preprocess an image
def load_and_preprocess_image(image_path):
    # Load the image (you might need to adjust this based on your actual data loading)
    image = load_image(image_path)
    # Preprocess the image (resize, normalize, etc.)
    preprocessed_image = preprocess_image(image)
    return preprocessed_image

# Placeholder functions for image loading and preprocessing
def load_image(image_path):
    # Implement image loading based on your dataset
    # (e.g., using OpenCV, PIL, or any other library)
    pass

def preprocess_image(image):
    # Implement image preprocessing steps (e.g., resizing, normalizing)
    pass

# Load and preprocess the dataset
data_dir = "/path/to/your/dataset"
features, labels = load_dataset(data_dir)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Initialize and train the Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")




# Assuming you have a new signature image
new_signature_path = "/path/to/new/signature.jpg"
new_signature_image = load_and_preprocess_image(new_signature_path)
new_signature_features = extract_hog_features(new_signature_image)

# Reshape the features array to be compatible with the model's input shape
new_signature_features = new_signature_features.reshape(1, -1)

# Use the trained model to make predictions
prediction = svm_model.predict(new_signature_features)

# Display the prediction
if prediction == 0:
    print("The signature is predicted to be genuine.")
else:
    print("The signature is predicted to be forged.")

